# Setup

## Verify we're in the Conda environment

In [ ]:
import sys

print(sys.executable)

## Import python packages

In [ ]:
import os
import sys
import json
import base64
import io
from dotenv import load_dotenv
import requests
import pprint
from pathlib import Path
import matplotlib.pyplot as plt
import subprocess
import jupyter_black
import pandas as pd
import matplotlib.dates as mdates

# Activate the jupyter_black extension, which reformats code cells with black
# https://github.com/n8henrie/jupyter-black
jupyter_black.load()

# Data

## Parameters

In [ ]:
WL_MAINERS_CREATED = 289
MAX_PUBLIC_MAINERS = 225
MAINERS_EXP1 = 82
MAX_MAINERS = WL_MAINERS_CREATED + MAX_PUBLIC_MAINERS + MAINERS_EXP1

ICP_WL_MAINER = 5
ICP_PUBLIC_MAINER = 10
ICP_MAINERS_EXP1 = 10

MAX_ICP_SPENT = (
    WL_MAINERS_CREATED * ICP_WL_MAINER
    + MAX_PUBLIC_MAINERS * ICP_PUBLIC_MAINER
    + MAINERS_EXP1 * ICP_MAINERS_EXP1
)

ICP_PRICE_USD = 5.1
MAX_USD_SPENT = MAX_ICP_SPENT * ICP_PRICE_USD

DAILY_BURN_RATE_LOW = 1
DAILY_BURN_RATE_MEDIUM = 2
DAILY_BURN_RATE_HIGH = 4
DAILY_BURN_RATE_VERY_HIGH = 6
DAILY_BURN_RATE = {
    "low": DAILY_BURN_RATE_LOW,
    "medium": DAILY_BURN_RATE_MEDIUM,
    "high": DAILY_BURN_RATE_HIGH,
    "very_high": DAILY_BURN_RATE_VERY_HIGH,
}

## Read data from get_mainers-prd.csv

to create this file, run

```bash
# from folder: funnAI
scripts/get_mainers.sh --network prd
```

In [ ]:
# Read data from CSV file
df = pd.read_csv("get_mainers-prd.csv")
# Convert columns to appropriate dtypes
df = df.iloc[3:].copy()  # Skip header rows if present in the data

df["timestamp"] = pd.to_datetime(df["timestamp"])
df["mainers_created"] = pd.to_numeric(df["mainers_created"], errors="coerce")

# Convert timestamp from UTC to California time
df["timestamp"] = df["timestamp"].dt.tz_convert("America/Los_Angeles")

## Calculate daily burn rate

In [ ]:
# filter df to include only rows where total_cycles is not 0
df2 = df[df["total_cycles"] != 0]

# For each row, calculate the daily_burn_rate using the formula:
# daily_burn_rate = 0.9 * (total_active_low*DAILY_BURN_RATE_LOW + total_active_medium*DAILY_BURN_RATE_MEDIUM + total_active_high*DAILY_BURN_RATE_HIGH + total_active_very_high*DAILY_BURN_RATE_VERY_HIGH)
df2["daily_burn_rate"] = (
    0.9
    * (
        df2["total_active_low"] * DAILY_BURN_RATE_LOW
        + df2["total_active_medium"] * DAILY_BURN_RATE_MEDIUM
        + df2["total_active_high"] * DAILY_BURN_RATE_HIGH
        + df2["total_active_very_high"] * DAILY_BURN_RATE_VERY_HIGH
    )
).astype(int)
df2

## Read ledger transactions

- download 'ledgers/ledger-icp-gamestate.csv' from https://dashboard.internetcomputer.org/account/300d6f0058417bb5131c7313a3fe7f7b90510ca2f413ab863d39b1e35eceebad

In [ ]:
GS_ICP_LEDGER_ACCOUNT = (
    "300d6f0058417bb5131c7313a3fe7f7b90510ca2f413ab863d39b1e35eceebad"
)

# Read gamestate icp ledger transactions from file 'ledgers/ledger-icp-gamestate.csv' into a pandas DataFrame
df_gs_icp_ledger = pd.read_csv("ledgers/ledger-icp-gamestate.csv")
# Convert timestamp to datetime
df_gs_icp_ledger["Timestamp"] = pd.to_datetime(df_gs_icp_ledger["Timestamp"])

df_gs_icp_ledger

# Plots

## Daily Burn Rate over time

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(7, 4), facecolor="black")
ax.set_facecolor("black")

# Plot the green curve with markers as dots
ax.plot(
    df2["timestamp"],
    df2["daily_burn_rate"],
    color="green",
    marker="o",
    linestyle="-",
    label="Daily Burn Rate (Trillions of Cycles) of funnAI protocol",
)

# Set x-axis ticks to actual data points
ax.set_xticks(df2["timestamp"])
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))

# Style the plot
ax.set_title("Daily Burn Rate of funnAI protocol", color="white", fontsize=16)
ax.set_xlabel("Date", color="white", fontsize=12)
ax.set_ylabel("Trillion Cycles", color="white", fontsize=12)
ax.tick_params(axis="x", colors="white", labelrotation=45, labelsize=10)
ax.tick_params(axis="y", colors="white", labelsize=10)
ax.grid(True, color="gray", linestyle="--", linewidth=0.5)

# Display the plot
fig.tight_layout()
plt.show()

## total_cycles locked over time

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(7, 4), facecolor="black")
ax.set_facecolor("black")

# Plot the green curve with markers as dots
ax.plot(
    df2["timestamp"],
    df2["total_cycles"],
    color="green",
    marker="o",
    linestyle="-",
    label="Total Cycles in mAIner canisters",
)

# Set x-axis ticks to actual data points
ax.set_xticks(df2["timestamp"])
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))

# Style the plot
ax.set_title("Trillions of Cycles in mAIner canisters", color="white", fontsize=16)
ax.set_xlabel("Date", color="white", fontsize=12)
ax.set_ylabel("Trillion Cycles", color="white", fontsize=12)
ax.tick_params(axis="x", colors="white", labelrotation=45, labelsize=10)
ax.tick_params(axis="y", colors="white", labelsize=10)
ax.grid(True, color="gray", linestyle="--", linewidth=0.5)

# Display the plot
fig.tight_layout()
plt.show()

## Active & Paused mAIners over time

In [ ]:
fig, ax2 = plt.subplots(figsize=(7, 4), facecolor="black")
ax2.set_facecolor("black")

bar_width = 0.3
x = df2["timestamp"]

active_low = df2["total_active_low"]
active_medium = df2["total_active_medium"]
active_high = df2["total_active_high"]
active_very_high = df2["total_active_very_high"]
paused = df2["total_paused"]

ax2.bar(
    x, active_low, width=bar_width, color="#045c04", label="Active Low", align="center"
)
ax2.bar(
    x,
    active_medium,
    width=bar_width,
    bottom=active_low,
    color="#1d741d",
    label="Active Medium",
    align="center",
)
ax2.bar(
    x,
    active_high,
    width=bar_width,
    bottom=active_low + active_medium,
    color="#2daf2d",
    label="Active High",
    align="center",
)
# ax2.bar(
#     x,
#     active_very_high,
#     width=bar_width,
#     bottom=active_low + active_medium + active_high,
#     color="#ffcc00",
#     label="Active Very High",
#     align="center",
# )
# ax2.bar(
#     x,
#     paused,
#     width=bar_width,
#     bottom=active_low + active_medium + active_high + active_very_high,
#     color="red",
#     label="Paused",
#     align="center",
# )
ax2.bar(
    x,
    paused,
    width=bar_width,
    bottom=active_low + active_medium + active_high,
    color="red",
    label="Paused",
    align="center",
)

ax2.set_xticks(df2["timestamp"])
ax2.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
ax2.set_title("Active & Paused mAIners over time", color="white", fontsize=16)
ax2.set_xlabel("Date", color="white", fontsize=12)
ax2.set_ylabel("Number of mAIners", color="white", fontsize=12)
ax2.tick_params(axis="x", colors="white", labelrotation=45, labelsize=10)
ax2.tick_params(axis="y", colors="white", labelsize=10)
ax2.grid(True, color="gray", linestyle="--", linewidth=0.5)
legend2 = ax2.legend(
    loc="upper left",
    bbox_to_anchor=(0.25, 0.75),
    fontsize=10,
    facecolor="black",
    edgecolor="white",
)

# flip order of legend items and set legend text color to white
handles, labels = ax2.get_legend_handles_labels()
handles = handles[::-1]
labels = labels[::-1]
legend2 = ax2.legend(
    handles,
    labels,
    loc="upper left",
    bbox_to_anchor=(0.05, 0.75),
    fontsize=10,
    facecolor="black",
    edgecolor="white",
)
for text in legend2.get_texts():
    text.set_color("white")

fig.tight_layout()

plt.show()

## WL Mainers Created Over Time

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(7, 4), facecolor="black")
ax.set_facecolor("black")

# Plot the green curve
ax.plot(
    df["timestamp"], df["mainers_created"], color="green", label="WL Mainers Created"
)

# Format x-axis ticks as mm-dd-hr
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))

# Style the plot
ax.set_title("WL Mainers Created Over Time", color="white", fontsize=16)
ax.set_xlabel("Date", color="white", fontsize=12)
ax.set_ylabel("WL Mainers Created", color="white", fontsize=12)
ax.tick_params(axis="x", colors="white", labelrotation=45, labelsize=10)
ax.tick_params(axis="y", colors="white", labelsize=10)
ax.grid(True, color="gray", linestyle="--", linewidth=0.5)

# Display the plot
fig.tight_layout()
plt.show()

## mAIners Created

In [ ]:
# Create the plot
fig, ax = plt.subplots(figsize=(7, 4), facecolor="black")
ax.set_facecolor("black")

# Format x-axis ticks as mm-dd
ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))

# Plot green curve where mainers_created <= WL_MAINERS_CREATED, blue above
created = df["mainers_created"]
timestamps = df["timestamp"] - pd.Timedelta(
    hours=7
)  # Adjust from UTC to PT (California)

# Find the split index where mainers_created first exceeds WL_MAINERS_CREATED
split_idx = (created > WL_MAINERS_CREATED).idxmax()
if created.max() <= WL_MAINERS_CREATED or split_idx == 0:
    # All values are below or equal to threshold, plot all green
    ax.plot(timestamps, created, color="green", label="Whitelist Mainers")
else:
    # Plot green segment
    ax.plot(
        timestamps.loc[: split_idx - 1],
        created.loc[: split_idx - 1],
        color="green",
        label="Whitelist Mainers",
    )
    # Plot blue segment
    ax.plot(
        timestamps.loc[split_idx - 1 :],
        created.loc[split_idx - 1 :],
        color="blue",
        label="Public Mainers",
    )

# Add a vertical, yellow dashed line at June 29, 12 PM PT
june_29_noon_pt = pd.Timestamp("2025-06-29 12:00:00", tz="America/Los_Angeles")
ax.axvline(
    x=june_29_noon_pt,
    color="yellow",
    linestyle="--",
    label="mAIning goes live",
)

# Add a vertical, yellow dotted line at July 10, 8 AM PT
# july_10_am_pt = pd.Timestamp("2025-07-10 05:00:00", tz="America/Los_Angeles")
# ax.axvline(
#     x=july_10_am_pt,
#     color="yellow",
#     linestyle=":",
#     label="Expansion sale goes live",
# )

# Add a red horizontal line at MAX_MAINERS = 600
# MAX_MAINERS = 600
# ax.axhline(
#     y=MAX_MAINERS,
#     color="red",
#     linestyle="--",
#     label=f"Max Mainers = {MAX_MAINERS}",
# )


# Add a legend, moved slightly down and to the right
legend = ax.legend(
    loc="upper left",
    bbox_to_anchor=(0.25, 0.75),
    fontsize=10,
    facecolor="black",
    edgecolor="white",
)
for text in legend.get_texts():
    text.set_color("white")

# Style the plot
ax.set_title("mAIners created", color="white", fontsize=16)
ax.set_xlabel("Date", color="white", fontsize=12)
ax.set_ylabel("funnAI Mainers Created", color="white", fontsize=12)
ax.tick_params(axis="x", colors="white", labelrotation=45, labelsize=10)
ax.tick_params(axis="y", colors="white", labelsize=10)
ax.grid(True, color="gray", linestyle="--", linewidth=0.5)

# Display the plot
fig.tight_layout()
plt.show()

## Cumulative ICP spent on creating mAIners at funnai.onicai.com

In [ ]:
# Calculate the number of new mainers at each step
new_mainers = created.diff().fillna(created.iloc[0]).astype(int)

# Create a boolean mask for WL and public mainers
is_wl = created <= WL_MAINERS_CREATED

# Calculate ICP spent at each step
icp_spent = new_mainers.where(is_wl, 0) * 5 + new_mainers.where(~is_wl, 0) * 10

# Cumulative ICP spent
cumulative_icp = icp_spent.cumsum()

# Plot
fig2, ax2 = plt.subplots(figsize=(7, 4), facecolor="black")
ax2.set_facecolor("black")

# Format x-axis ticks as mm-dd
ax2.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))

# Plot green for WL, blue for public
if created.max() <= WL_MAINERS_CREATED or split_idx == 0:
    ax2.plot(timestamps, cumulative_icp, color="green", label="WL ICP Spent")
else:
    ax2.plot(
        timestamps.loc[: split_idx - 1],
        cumulative_icp.loc[: split_idx - 1],
        color="green",
        label="White Listed mAIners",
    )
    ax2.plot(
        timestamps.loc[split_idx - 1 :],
        cumulative_icp.loc[split_idx - 1 :],
        color="blue",
        label="Public Sale mAIners",
    )

# # Add a red horizontal line at max ICP spent
# ax2.axhline(
#     y=MAX_ICP_SPENT,
#     color="red",
#     linestyle="--",
#     label=f"Max ICP available mAIners = {MAX_ICP_SPENT} ICP",
# )

# Add a vertical, yellow dashed line at June 29, 12 PM PT
june_29_noon_pt = pd.Timestamp("2025-06-29 12:00:00", tz="America/Los_Angeles")
ax2.axvline(
    x=june_29_noon_pt,
    color="yellow",
    linestyle="--",
    label="mAIning goes live",
)

# Add a vertical, yellow dotted line at July 10, 8 AM PT
# july_10_am_pt = pd.Timestamp("2025-07-10 08:00:00", tz="America/Los_Angeles")
# ax2.axvline(
#     x=july_10_am_pt,
#     color="yellow",
#     linestyle=":",
#     label="Expansion sale (82) goes live",
# )

ax2.set_title(
    "Cumulative ICP spent on creating mAIners",
    color="white",
    fontsize=16,
)
ax2.set_xlabel("Date", color="white", fontsize=12)
ax2.set_ylabel("ICP Spent", color="white", fontsize=12)
ax2.tick_params(axis="x", colors="white", labelrotation=45, labelsize=10)
ax2.tick_params(axis="y", colors="white", labelsize=10)
ax2.grid(True, color="gray", linestyle="--", linewidth=0.5)
legend2 = ax2.legend(
    loc="upper left",
    bbox_to_anchor=(0.25, 0.65),
    fontsize=10,
    facecolor="black",
    edgecolor="white",
)
for text in legend2.get_texts():
    text.set_color("white")
fig2.tight_layout()
plt.show()

## Cumulative USD spent on creating mAIners at funnai.onicai.com

In [ ]:
# Calculate cumulative spend in USD
cumulative_usd = cumulative_icp * ICP_PRICE_USD

# Plot cumulative USD spent
fig3, ax3 = plt.subplots(figsize=(7, 4), facecolor="black")
ax3.set_facecolor("black")

# Format x-axis ticks as mm-dd
ax3.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))

# Plot green for WL, blue for public
if created.max() <= WL_MAINERS_CREATED or split_idx == 0:
    ax3.plot(timestamps, cumulative_usd, color="green", label="Whitelist mAIners")
else:
    ax3.plot(
        timestamps.loc[: split_idx - 1],
        cumulative_usd.loc[: split_idx - 1],
        color="green",
        label="Whitelist mAIners",
    )
    ax3.plot(
        timestamps.loc[split_idx - 1 :],
        cumulative_usd.loc[split_idx - 1 :],
        color="blue",
        label="Public Sale mAIners",
    )

# Add a red horizontal line at max
# ax3.axhline(
#     y=MAX_USD_SPENT,
#     color="red",
#     linestyle="--",
#     label=f"Max USD Spent on available mAIners = ${MAX_USD_SPENT}",
# )

# Add a vertical, yellow dashed line at June 29, 12 PM PT
june_29_noon_pt = pd.Timestamp("2025-06-29 12:00:00", tz="America/Los_Angeles")
ax3.axvline(
    x=june_29_noon_pt,
    color="yellow",
    linestyle="--",
    label="mAIning goes live",
)

# Add a vertical, yellow dotted line at July 10, 8 AM PT
# july_10_am_pt = pd.Timestamp("2025-07-10 08:00:00", tz="America/Los_Angeles")
# ax3.axvline(
#     x=july_10_am_pt,
#     color="yellow",
#     linestyle=":",
#     label="Expansion sale (82) goes live",
# )

print(f"Total mAIners created: {created.iloc[-1]}")
print(f"Total mAIners remaining: {MAX_MAINERS - created.iloc[-1]}")
print(f"Total ICP spent: {cumulative_icp.iloc[-1]}")
print(f"Total USD spent: {cumulative_usd.iloc[-1]}")

ax3.set_title(
    "Cumulative USD spent on creating mAIners",
    color="white",
    fontsize=16,
)
ax3.set_xlabel("Date", color="white", fontsize=12)
ax3.set_ylabel("USD Spent ($)", color="white", fontsize=12)
ax3.tick_params(axis="x", colors="white", labelrotation=45, labelsize=10)
ax3.tick_params(axis="y", colors="white", labelsize=10)
ax3.grid(True, color="gray", linestyle="--", linewidth=0.5)
legend3 = ax3.legend(
    loc="upper left",
    bbox_to_anchor=(0.25, 0.65),
    fontsize=10,
    facecolor="black",
    edgecolor="white",
)
for text in legend3.get_texts():
    text.set_color("white")
fig3.tight_layout()
plt.show()

## Cumulative ICP for mAIners & Top Ups

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Filter and sort
received_mask = (df_gs_icp_ledger["To"] == GS_ICP_LEDGER_ACCOUNT) & (
    df_gs_icp_ledger["Amount"] > 0
)
df_received = df_gs_icp_ledger.loc[received_mask].sort_values("Timestamp")

# Compute cumulative received
df_received["Cumulative_ICP"] = df_received["Amount"].cumsum()

# Plotting
fig, ax = plt.subplots(figsize=(7, 4), facecolor="black")
ax.set_facecolor("black")

ax.plot(
    df_received["Timestamp"],
    df_received["Cumulative_ICP"],
    color="green",
    label="ICP",
)

ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
ax.set_title("Cumulative ICP for funnAI mAIners & topups", color="white", fontsize=16)
ax.set_xlabel("Date", color="white", fontsize=12)
ax.set_ylabel("ICP", color="white", fontsize=12)
ax.tick_params(axis="x", colors="white", labelrotation=45, labelsize=10)
ax.tick_params(axis="y", colors="white", labelsize=10)
ax.grid(True, color="gray", linestyle="--", linewidth=0.5)

# legend = ax.legend(
#     loc="upper left",
#     bbox_to_anchor=(0.15, 0.85),
#     fontsize=10,
#     facecolor="black",
#     edgecolor="white",
# )
for text in legend.get_texts():
    text.set_color("white")

fig.tight_layout()
plt.show()